In [1]:
import keras
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JLXO-4-TMJB87CTQKc6dVclYtSXBMueJZxQKcaRUK0VP',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfiredetection-donotdelete-pr-bpytmsf9pwiglr'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=180,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
x_train = train_datagen.flow_from_directory(r'./Dataset/train_set/',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 436 images belonging to 2 classes.


In [14]:
x_test = train_datagen.flow_from_directory(r'./Dataset/test_set/',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 121 images belonging to 2 classes.


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten


In [16]:
model = Sequential()
model.add(Convolution2D(32, (3,3), input_shape=(128, 128, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(150,activation="relu"))
model.add(Dense(1, activation="sigmoid"))


In [17]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [18]:
model.fit(x_train, steps_per_epoch=14, epochs=10, validation_data=x_test, validation_steps=4)

Epoch 1/10
14/14 [==============================] - 24s 2s/step - loss: 2.0141 - accuracy: 0.7133 - val_loss: 0.1630 - val_accuracy: 0.9421
Epoch 2/10
14/14 [==============================] - 22s 2s/step - loss: 0.3240 - accuracy: 0.8922 - val_loss: 0.1051 - val_accuracy: 0.9835
Epoch 3/10
14/14 [==============================] - 22s 2s/step - loss: 0.2306 - accuracy: 0.9014 - val_loss: 0.1186 - val_accuracy: 0.9421
Epoch 4/10
14/14 [==============================] - 21s 2s/step - loss: 0.1938 - accuracy: 0.9174 - val_loss: 0.0852 - val_accuracy: 0.9752
Epoch 5/10
14/14 [==============================] - 22s 1s/step - loss: 0.1953 - accuracy: 0.9243 - val_loss: 0.1242 - val_accuracy: 0.9339
Epoch 6/10
14/14 [==============================] - 21s 2s/step - loss: 0.1797 - accuracy: 0.9128 - val_loss: 0.0790 - val_accuracy: 0.9835
Epoch 7/10
14/14 [==============================] - 21s 1s/step - loss: 0.1688 - accuracy: 0.9335 - val_loss: 0.0905 - val_accuracy: 0.9421
Epoch 8/10
14/14 [==

Save the model

In [19]:
model.save("model.h5")

In [20]:
!tar -zcvf model.tgz model.h5

model.h5


Prediction

In [21]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 18.1 MB/s eta 0:00:01


In [22]:
from ibm_watson_machine_learning import APIClient

API_KEY = "5W65wtnV1kus0WCtJ4HEMzw-lVetPUlY_B2Nje3fDq4p"

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(credentials)

In [24]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [32]:
 space = client.spaces.get_details()
 space

{'resources': []}

In [33]:
space_uid = guid_from_space_name(client, 'Forest fire detection')
print("Space UID: ", space_uid)

Space UID:  d9308ab8-179a-48da-974b-d986f1649bd5


In [34]:
client.set.default_space(space_uid)

'SUCCESS'

In [35]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [36]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [37]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'